<a href="https://colab.research.google.com/github/gomyk/NNStudy/blob/moonwon/%5BRL%5D%5BMW%5DAcrobot-V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)

import math
import glob
import io
import base64
from IPython.display import HTML

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400,900),)
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1037'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1037'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

HIDDEN = 32
BATCH_SIZE = 32
PERCENTILE = 50
LIMIT = 250
EPISODE = 100
class Net(nn.Module):
  def __init__(self, obs_size, hidden, action_size):
    super(Net, self).__init__()
    self.net=nn.Sequential(
        nn.Linear(obs_size, hidden),
        nn.ReLU(),
        nn.Linear(hidden, action_size)
    )

  def forward(self, x):
    return self.net(x)

In [0]:
import copy

def get_batch(batch_size, env, net):
  print('')
  batch = []
  reward = 0.0
  obs = env.reset()
  steps = []
  sm = nn.Softmax(dim=1)
  while True:
    obs_v = torch.FloatTensor([obs])
    obs_v = obs_v.to(device)
    action_v = sm(net(obs_v)).cpu().data.numpy()[0]
    action = np.random.choice(env.action_space.n, p=action_v)
    next_obs, rew, done, _ = env.step(action)
    reward += rew
    steps.append((obs,action))
    if done:
      batch.append((reward, steps))
      reward = 0.0
      next_obs = env.reset()
      steps = []
      if len(batch) == batch_size:
        yield batch
        obs = env.reset()
        batch=[]
    obs = next_obs

In [0]:
def filter_batch(batch):
  rews = list(map(lambda episode: episode[0],batch))
  rews_threshold = np.percentile(rews, PERCENTILE)
  rews_mean = np.mean(rews)

  ret_action = []
  ret_obs = []

  for scene in batch:
    if scene[0] < rews_threshold:
      continue
    ret_obs.extend(map(lambda step: step[0], scene[1]))
    ret_action.extend(map(lambda step: step[1], scene[1]))

  return torch.FloatTensor(ret_obs), torch.LongTensor(ret_action), rews_threshold, rews_mean

In [0]:
def evalNet(env, net):
  #Best 100 episode rewards
  sm = nn.Softmax(dim=1)
  total_reward = 0
  for i in range(EPISODE):
    obs = env.reset()
    reward = 0
    while True:
      obs_v = torch.FloatTensor([obs]).to(device)
      action = sm(net(obs_v))
      _, predicted = torch.max(action.data, 1)
      next_obs, rew, done, _ = env.step(predicted)
      reward += rew
      obs = next_obs
      if done:
        total_reward += reward
        break;
  return total_reward/EPISODE

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    for mp4 in mp4list:
      video = io.open(mp4, 'r+b').read()
      print(mp4)
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 300px;">
                                  <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
                                  </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")


In [0]:
env = gym.wrappers.Monitor(gym.make("Acrobot-v1"), "video", force=True, video_callable=lambda c:c%100 ==0)
net = Net(env.observation_space.shape[0], HIDDEN, env.action_space.n)
CELoss = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), 0.01)
device = torch.device("cuda")
net = net.to(device)

for i,batch in enumerate(get_batch(BATCH_SIZE, env, net)):
  obs, action, threshold, mean = filter_batch(batch)
  opt.zero_grad()
  obs = obs.to(device)
  action = action.to(device)
  action_out = net(obs)
  loss = CELoss(action_out, action)
  loss.backward()
  opt.step()
  print("%d, loss = %.4f, th = %.2f, mean = %.2f "%(i, loss, threshold, mean))
  if(i == 10) :
    print("%d EVAL : %.4f"%(i, evalNet(env, net)))
  if(i == EPISODE) :
      show_video()
  
    


0, loss = 1.0885, th = -500.00, mean = -500.00 
1, loss = 1.0768, th = -500.00, mean = -499.94 
2, loss = 1.0635, th = -500.00, mean = -494.31 
3, loss = 1.0430, th = -500.00, mean = -458.78 
